# INFORMATION RETRIEVAL SYSTEM

###Import libraries

In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import OrderedDict
import contractions
import requests
import os
import pandas as pd
import numpy
from bs4 import BeautifulSoup
from gensim.models import Word2Vec

nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bomme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bomme\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Extract data process

 - From the UNT website filter all the links using web scraping library beautifil soup.
 - Search the list of valid links for nested links as well.
 - Filter the text of the links and perform text processing.


In [2]:
URL = "https://www.unt.edu/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [3]:
def clean_text(txt):
  txt = [word.strip('''@^#!$()-{}%;:,.<>?&"\_/*~''') for word in txt]
  txt = [word.lower() for word in txt]
  return(txt)

In [4]:
all_links = soup.find_all("a")
source_urls = []
train_data = []
for link in all_links:
  try:
    if(str(link.get("href")).startswith("http") and link.get('href') not in source_urls):
      if(link.text and len(link.text)>1):
        content = contractions.fix(link.text)
        source_urls.append(link.get("href"))
        word_token = word_tokenize(content)
        word_token = clean_text(word_token)
        train_data.append(word_token)
      else:
        pass
  except:
    print("Invalid!")



In [5]:
df1 = pd.DataFrame({'text':train_data,'url':source_urls})

In [6]:
URL = ['https://www.unt.edu/']
source_urls_extra = []
train_data_extra = []
for urls in df1.url[0:100]:
    try:
      page = requests.get(urls)
      soup = BeautifulSoup(page.content, "html.parser")
      all_links = soup.find_all("a")
      for link in all_links:
        try:
          if(str(link.get("href")).startswith("http") and link.get('href') not in source_urls_extra):
            if(link.text and len(link.text)>1):
              content = contractions.fix(link.text)
              source_urls_extra.append(link.get("href"))
              word_token = word_tokenize(content)
              word_token = clean_text(word_token)
              train_data_extra.append(word_token)
            else:
              pass
        except:
          print("Invalid!")
    except:
        pass
  
df2 = pd.DataFrame({'text':train_data_extra,'url':source_urls_extra})

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [7]:
superdf = pd.concat([df1,df2])

In [8]:
# Training a word2vec model from the given data set
w2v_model = Word2Vec(min_count=1,window=5, sg=1,workers=4)

In [9]:
w2v_model.build_vocab(superdf.text)

In [10]:
w2v_model.train(superdf.text,total_examples=w2v_model.corpus_count,epochs=200)

(675713, 890400)

In [11]:
w2v_model.wv.most_similar('unt')

[('discover', 0.4882465600967407),
 ('google', 0.4737807810306549),
 ('dataanalytics.unt.edu', 0.4462285041809082),
 ('at', 0.44399288296699524),
 ('nbc', 0.44246792793273926),
 ('facilities', 0.44187384843826294),
 ('insecurity', 0.4274078607559204),
 ('alumni', 0.4273844063282013),
 ('intensive', 0.414808988571167),
 ('frisco', 0.413099080324173)]

In [12]:
def search_db(keyword):
  flag = -1
  counter = 0
  indexed_counter = []
  for i in superdf.text:
    if(len(i)>1):
      for words in i:
        if(words == keyword):
          flag = i
          indexed_counter.append(counter)
    else:
      if(i == keyword):
          flag = i
          indexed_counter.append(counter)
    counter = counter +1
  return(indexed_counter)
  

In [15]:
#Search Engine

keyword = str(input('Enter a string to search :'))

try:
  print("Searching the database for most suitable result!")
  results = search_db(w2v_model.wv.most_similar(keyword)[0][0])
  for i in results:
    print(superdf.iloc[i].url)
except:
  print("The word was not found! (CONTEXT ERROR)")

Enter a string to search :admissions
Searching the database for most suitable result!
http://admissions.unt.edu/freshman/how-to-apply


In [14]:
df2

text  \
0                                        [unt, map]   
1                            [visitor, information]   
2                                        [unt, map]   
3     [enable, javascript, to, see, google, maps, ]   
4                                [unt, mobile, app]   
...                                             ...   
1433                    [electronic, accessibility]   
1434                      [compliance, trust, line]   
1435           [requests, for, public, information]   
1436          [institutional, finance, , 51.9741, ]   
1437                        [institutional, resume]   

                                                    url  
0                                 http://bit.ly/2eOPgoh  
1                         https://www.unt.edu/community  
2                       https://goo.gl/maps/gmkVRXCJqCq  
3     https://support.google.com/maps/?hl=en&authuse...  
4                                http://mobile.unt.edu/  
...                                                 ...  
1433  https://untsystem.policytech.com/dotNet/docume...  
1434  https://secure.ethicspoint.com/domain/media/en...  
1435  https://untsystem.policytech.com/dotNet/docume...  
1436  https://www.untsystem.edu/reports-and-data/ind...  
1437        https://www.thecb.state.tx.us/apps/resumes/  

[1438 rows x 2 columns]